In [ ]:
import sys
import cv2
import imageio
import easygui
import os
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image


In [ ]:
def upload():
    imagepath=easygui.fileopenbox()#this will open box to choose file and store filepath as string
    cartoonify(imagepath)
    

In [ ]:
def cartoonify(path):
    oimage=cv2.imread(path)#TO STORE IMAGE IN FORM OF NO. array
    oimage=cv2.cvtColor(oimage,cv2.COLOR_BGR2RGB)#in rgb format
    if(oimage is None):
        print("Choose Appropriate File.")
        sys.exit()
    resized1=cv2.resize(oimage,(960,540))
    greyimage=cv2.cvtColor(oimage,cv2.COLOR_BGR2GRAY)#converting in graysale
    resized2=cv2.resize(greyimage,(960,540))#resizing
    smoothgreyimage= cv2.medianBlur(greyimage,5)#using 5 as kernel size for median blurring
    resized3=cv2.resize(smoothgreyimage,(960,540))
    getedge=cv2.adaptiveThreshold(smoothgreyimage,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,9,9)
    #parameters and constnat like max_value is in range of 0-255,last two are blocksize(usually 3,5,7...) and
    #last one is constant which is subtrated from mean geenrally +ve can be -ve or 0
    resized4=cv2.resize(getedge,(960,540))
    colorimage=cv2.bilateralFilter(oimage,9,300,300)#obtaining lightened color image of original
    resized5=cv2.resize(colorimage,(960,540))
    cartoonimage = cv2.bitwise_and(colorimage, colorimage, mask=getedge)#masking edged & smooth image
    resized6=cv2.resize(cartoonimage,(960,540))
    #plotting all the transitions of image
    images=[resized1,resized2,resized3,resized4,resized5,resized6]
    fig, axes = plt.subplots(3,2, figsize=(8,8), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i], cmap='gray')
    save1=Button(top,text="Save cartoonified Image",command=lambda:save(resized6,path),padx=0,pady=5)
    save1.configure(background='#364156', foreground='white',font=('calibri',10,'bold'))
    save1.pack(side=TOP,pady=50)
    plt.show()
    
    

                          

In [ ]:
def save(resized6,imagepath):
    newname="cartoonified_img"
    path1=os.path.dirname(imagepath)
    extension=os.path.splitext(imagepath)[1]
    path2=os.path.join(path1,newname+extension)
    cv2.imwrite(path2,cv2.cvtColor(resized6,cv2.COLOR_RGB2BGR))
    msg="Cartoonified Image saved by name"+newname+" at "+path2
    tk.messagebox.showinfo(ttile=None,message=msg)


In [ ]:
top=tk.Tk()
top.geometry('400x400')
top.title('cartoonify your Image')
top.configure(background='white')
label=Label(top,background='#CDCDCD',font=('calibri',20,'bold'))
upload=Button(top,text="cartoonify the image",command=upload,padx=10,pady=5)
upload.configure(background="#364156",foreground="white",font=('calibri',10,'bold'))
upload.pack(side=TOP,pady=50)

In [ ]:
# to run the mainloop upload the picture and obtain cartoonified image 
top.mainloop()